In [1]:
import secrets
import time
import math


In [2]:
def randbits (n_bits :int )->int :
    return secrets .randbits (n_bits )

def rand_odd (n_bits :int )->int :

    x =secrets .randbits (n_bits -1 )|(1 <<(n_bits -1 ))
    return x |1

SMALL_PRIMES =[
3 ,5 ,7 ,11 ,13 ,17 ,19 ,23 ,29 ,31 ,
37 ,41 ,43 ,47 ,53 ,59 ,61 ,67 ,71 ,73 ,
79 ,83 ,89 ,97
]

def trial_division_small (n :int )->bool :

    for p in SMALL_PRIMES :
        if n ==p :
            return True
        if n %p ==0 :
            return False
    return True


In [3]:
def is_probable_prime_fermat (n :int ,k :int =10 )->bool :
    if n <2 :
        return False
    small =[2 ,3 ,5 ]
    if n in small :
        return True
    if n %2 ==0 :
        return False
    if not trial_division_small (n ):
        return False

    for _ in range (k ):
        a =secrets .randbelow (n -3 )+2
        if math .gcd (a ,n )!=1 :
            return False
        if pow (a ,n -1 ,n )!=1 :
            return False
    return True


In [4]:
def jacobi (a :int ,n :int )->int :
    if n <=0 or n %2 ==0 :
        raise ValueError ("n must be a positive odd integer")
    a %=n
    result =1
    while a !=0 :
        while a %2 ==0 :
            a //=2
            r =n %8
            if r ==3 or r ==5 :
                result =-result
        a ,n =n ,a
        if a %4 ==3 and n %4 ==3 :
            result =-result
        a %=n
    return result if n ==1 else 0


In [5]:
def is_probable_prime_solovay_strassen (n :int ,k :int =10 )->bool :
    if n <2 :
        return False
    if n in (2 ,3 ):
        return True
    if n %2 ==0 :
        return False
    if not trial_division_small (n ):
        return False

    for _ in range (k ):
        a = secrets.randbelow (n -3 )+2
        g = math.gcd (a ,n )
        if g !=1 :
            return False
        j = jacobi (a ,n )
        if j ==0 :
            return False
        x =pow (a ,(n -1 )//2 ,n )

        j_mod =j %n
        if x !=j_mod :
            return False
    return True


In [6]:
def is_probable_prime_miller_rabin (n :int ,k :int =10 )->bool :
    if n <2 :
        return False
    small =[2 ,3 ,5 ,7 ,11 ,13 ,17 ,19 ,23 ,29 ,31 ,37 ]
    if n in small :
        return True
    if n %2 ==0 :
        return False
    if not trial_division_small (n ):
        return False


    d =n -1
    s =0
    while d %2 ==0 :
        d //=2
        s +=1

    for _ in range (k ):
        a =secrets .randbelow (n -3 )+2
        x =pow (a ,d ,n )
        if x ==1 or x ==n -1 :
            continue
        witness =True
        for _r in range (s -1 ):
            x =(x *x )%n
            if x ==n -1 :
                witness =False
                break
        if witness :
            return False
    return True


In [7]:
def generate_prime_chebyshev (n_bits :int ,k :int =10 )->int :

    x =rand_odd (n_bits )
    while True :
        if is_probable_prime_miller_rabin (x ,k =k ):
            return x
        x +=2


In [8]:
tests ={
"Fermat":is_probable_prime_fermat ,
"Solovay–Strassen":is_probable_prime_solovay_strassen ,
"Miller–Rabin":is_probable_prime_miller_rabin ,
}

samples =[
(2 ,True ),
(3 ,True ),
(5 ,True ),
(17 ,True ),
(19 ,True ),
(21 ,False ),
(221 ,False ),
(561 ,False ),
]

for n ,expected_prime in samples :
    print (f"n={n} (expected prime={expected_prime})")
    for name ,fn in tests .items ():

        res =fn (n ,k =10 )
        print (f"  {name:16s}: {res}")
    print ()


n=2 (expected prime=True)
  Fermat          : True
  Solovay–Strassen: True
  Miller–Rabin    : True

n=3 (expected prime=True)
  Fermat          : True
  Solovay–Strassen: True
  Miller–Rabin    : True

n=5 (expected prime=True)
  Fermat          : True
  Solovay–Strassen: True
  Miller–Rabin    : True

n=17 (expected prime=True)
  Fermat          : True
  Solovay–Strassen: True
  Miller–Rabin    : True

n=19 (expected prime=True)
  Fermat          : True
  Solovay–Strassen: True
  Miller–Rabin    : True

n=21 (expected prime=False)
  Fermat          : False
  Solovay–Strassen: False
  Miller–Rabin    : False

n=221 (expected prime=False)
  Fermat          : False
  Solovay–Strassen: False
  Miller–Rabin    : False

n=561 (expected prime=False)
  Fermat          : False
  Solovay–Strassen: False
  Miller–Rabin    : False



In [9]:
def error_bounds (k :int )->dict :
    return {
    "Solovay–Strassen":2 **(-k ),
    "Miller–Rabin":4 **(-k ),
    }

for k in (5 ,10 ,20 ):
    b =error_bounds (k )
    print (f"k={k}:  SS<= {b['Solovay–Strassen']:.3e},  MR<= {b['Miller–Rabin']:.3e}  (Fermat: no universal bound)")


k=5:  SS<= 3.125e-02,  MR<= 9.766e-04  (Fermat: no universal bound)
k=10:  SS<= 9.766e-04,  MR<= 9.537e-07  (Fermat: no universal bound)
k=20:  SS<= 9.537e-07,  MR<= 9.095e-13  (Fermat: no universal bound)


In [10]:
def make_composite_same_bits (n_bits :int )->int :
    x =rand_odd (n_bits )

    r =x %3
    if r !=0 :
        x +=(3 -r )
    if x %2 ==0 :
        x +=3

    if x .bit_length ()>n_bits :
        x -=6
    if x ==3 :
        x +=6
    return x

def time_ms (fn ,*args ,repeats :int =5 ,warmup :int =1 ,**kwargs )->float :

    for _ in range (warmup ):
        fn (*args ,**kwargs )
    times =[]
    for _ in range (repeats ):
        t0 =time .perf_counter ()
        fn (*args ,**kwargs )
        t1 =time .perf_counter ()
        times .append ((t1 -t0 )*1000.0 )
    return sum (times )/len (times )

def benchmark (bits_list =(1024 ,2048 ,4096 ),k_list =(5 ,10 ,20 ),repeats =5 ):
    for n_bits in bits_list :
        print ("="*70 )
        print (f"{n_bits}-bit numbers")
        prime =generate_prime_chebyshev (n_bits ,k =10 )
        comp =make_composite_same_bits (n_bits )

        print ("Samples:")
        print (f"  prime bit_length={prime.bit_length()},  composite bit_length={comp.bit_length()}")
        print ()

        for k in k_list :
            print (f"k={k}")
            for name ,fn in tests .items ():
                t_prime =time_ms (fn ,prime ,k =k ,repeats =repeats )
                t_comp =time_ms (fn ,comp ,k =k ,repeats =repeats )
                print (f"  {name:16s}: prime={t_prime:.3f} ms, composite={t_comp:.3f} ms")
            print ()


benchmark (bits_list =(1024 ,2048 ,4096 ),k_list =(5 ,10 ,20 ),repeats =3 )


1024-bit numbers
Samples:
  prime bit_length=1024,  composite bit_length=1024

k=5
  Fermat          : prime=18.752 ms, composite=0.001 ms
  Solovay–Strassen: prime=25.182 ms, composite=0.001 ms
  Miller–Rabin    : prime=20.205 ms, composite=0.001 ms

k=10
  Fermat          : prime=38.587 ms, composite=0.001 ms
  Solovay–Strassen: prime=48.766 ms, composite=0.002 ms
  Miller–Rabin    : prime=40.980 ms, composite=0.002 ms

k=20
  Fermat          : prime=78.531 ms, composite=0.001 ms
  Solovay–Strassen: prime=83.524 ms, composite=0.001 ms
  Miller–Rabin    : prime=85.398 ms, composite=0.001 ms

2048-bit numbers
Samples:
  prime bit_length=2048,  composite bit_length=2048

k=5
  Fermat          : prime=128.802 ms, composite=0.002 ms
  Solovay–Strassen: prime=133.162 ms, composite=0.001 ms
  Miller–Rabin    : prime=129.840 ms, composite=0.002 ms

k=10
  Fermat          : prime=262.487 ms, composite=0.002 ms
  Solovay–Strassen: prime=274.422 ms, composite=0.001 ms
  Miller–Rabin    : prime=